PEFT and bitsandbytes to finetune LORA checkpoint

In [3]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (

In [35]:
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes

Found existing installation: bitsandbytes 0.45.0
Uninstalling bitsandbytes-0.45.0:
  Successfully uninstalled bitsandbytes-0.45.0
  Using cached bitsandbytes-0.45.0-py3-none-manylinux_2_24_x86_64.whl.metadata (2.9 kB)
Using cached bitsandbytes-0.45.0-py3-none-manylinux_2_24_x86_64.whl (69.1 MB)


In [2]:
import torch
import bitsandbytes as bnb
print(bnb.__version__)
print(torch.cuda.is_available())

0.45.0
True


In [25]:
!pip install -U transformers

In [3]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

12.1
True


In [4]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

config = BitsAndBytesConfig(load_in_8bit=True)
try:
    model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-560m", quantization_config=config, device_map="auto")
    print("Model loaded successfully with 8-bit quantization!")
except ImportError as e:
    print("Quantization ImportError:", e)
except Exception as e:
    print("Other error:", e)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Model loaded successfully with 8-bit quantization!


In [33]:
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [18]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   32G   82G  28% /
tmpfs            64M     0   64M   0% /dev
shm             5.7G  4.0K  5.7G   1% /dev/shm
/dev/root       2.0G  1.2G  820M  59% /usr/sbin/docker-init
tmpfs           6.4G   23M  6.4G   1% /var/colab
/dev/sda1        86G   58G   29G  67% /kaggle/input
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [19]:
!du -sh ~/.cache/*  # Check size of subdirectories

52K	/root/.cache/huggingface
36K	/root/.cache/matplotlib
56M	/root/.cache/node-gyp
70M	/root/.cache/pip
27M	/root/.cache/uv


In [20]:
# HERE WE ARE TRAINING THE ADAPTERS NOT THE original  WEIGHTS
# We will be adding weights to the models are various points and fine-tuning those weights
# We will set up the huggingface hub at the start so that we can later save our weights to HF hub
# the token is a write token which is saved by the name - PEFT-LORA-CHECKPOINTS
# the actual token is saved in the environment variable HF_TOKEN
from huggingface_hub import notebook_login
notebook_login()

Token has not been saved to git credential helper.


In [21]:
!nvidia-smi -L
# if nvidia-smi doesn't work , use the T4 and make the model we are
# using to be smaller version of the bloom model

GPU 0: Tesla T4 (UUID: GPU-23755a2c-e0ff-0c6f-02c1-80fad7431e39)


SETTING UP THE MODEL

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
# bitsandbytes from HF turns your model into 8 bits
# Model won't take up a lot of GPU ram
# Makes it easier and faster to store
import bitsandbytes as bnb
from transformers import AutoTokenizer , AutoConfig , AutoModelForCausalLM,BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Enable 8-bit quantization
    llm_int8_threshold=6.0  # Default value; adjust if necessary
)
model  = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloomz-560m" ,
    quantization_config=quantization_config,
    # if you have multiple devices[multiple-gpus] , you can do a device-map parts of the model
    # accross
    device_map ="auto"
)

tokenizer  = AutoTokenizer.from_pretrained("bigscience/bloomz-560m")

# WARNINGS: The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions.
# Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
# Stopped running due to model size - see if you can find smaller model

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [2]:
# print(model)
# [FROM THE LORA PAPER]
# QUERY_KEY_VALUE IN BLOOM -> Choose this weight to decompose and learn the decompositions of .
# If the model was something from the LLAMA family , then we will have proj_v and proj_q modules to decompose and fine-tune on

#


FREEZE THE ORIGINAL WEIGHTS


In [6]:
# FOLLOWING IS BASICALLY PRE-PROCESSING THAT HELPS IMPROVE STABILITY
for param in model.parameters():
  # freeze the model - train adapters later
  param.requires_grad = False
  if param.ndim==1:
    # cast the small params[dim=1] (eg.layernorm) to float32 for stability
    param.data = param.data.to(torch.float32)


#reduce number of stored activations
model.gradient_checkpointing_enable()
# enabling input gradients
model.enable_input_require_grads()


# cast the numerical output to float32 for stability , wrapper around sequential layers
class CastOutputToFloat(nn.Sequential):
  def forward(self,x):
    return super().forward(x).to(torch.float32)

# lm_head is the model's outout layer , wrap it with castoutputtofloat func,
#  Ensures that the logits (outputs of the lm_head) are cast to float32, making them
#  more stable for tasks like computing loss or metrics during training.
model.lm_head = CastOutputToFloat(model.lm_head)


# In PyTorch, the requires_grad attribute of a tensor indicates whether or not
# PyTorch should compute gradients for that tensor during the backward pass.
# requires_grad=True: Enables gradient computation for the tensor.
# When requires_grad=False, PyTorch does not track operations, so no gradients are computed,
# and the tensor is treated as a constant.
#  It is used To specify which parameters in a model should be updated during training.
# To save memory and computation time by disabling gradient tracking for parameters that don't need to be updated.

# 1D params are 1D tensors
# ndim==1 checks if the parameter is a 1D tensor (like biases or weights in layer normalization) and casts it to float32.
# In mixed precision training (e.g., using float16 for efficiency), small parameters like biases or layer norm weights can cause numerical instability.
#  Converting these parameters to float32 helps ensure stability without significantly affecting performance.

# Reducing Stored Activations : Enables gradient checkpointing, which reduces the memory usage for activations during training by
# recomputing them during backpropagation instead of storing them.
# This trade-off between computation and memory usage is particularly useful for training large models with limited GPU memory.


#  Enabling Input Gradients: This ensures that gradients with respect to the model’s inputs are computed.
# Necessary when applying techniques like LoRA (Low-Rank Adaptation) or other methods that modify intermediate layers during fine-tuning.
#  This enables the model to propagate gradients correctly for such custom layers or inputs.





In [7]:
# !rm -rf ~/.cache/huggingface

# above command removes the hf cache
# I have used it to clear out the bloom larger model that i downloaded




Setting up LORA ADAPTERS

In [8]:
# THIS IS THE HELPER FUNC

# The purpose of the function is to calculate and print the number of trainable parameters versus the total number of parameters in the model.
# _: The name of the parameter is ignored in this loop since it is not needed.
# param: Represents the actual parameter tensor.
# numel stands for number of elements
# For a tensor with a specific shape, .numel() computes the product of its dimensions to determine how many elements the tensor contains.


def print_trainable_parameters(model):
  trainable_params=0
  all_params = 0
  for _,param in model.named_parameters():
    all_params += param.numel()
    if param.requires_grad:
  # if we are setting requires_grad = True , that means we are computing backpropogation for those params, hence they will be included in training

      trainable_params += param.numel()
    print(f"trainable params:{trainable_params} || all_params:{all_params} || trainable% :{100*trainable_params/all_params}")


'''
Why Use .numel()? : Returns total number of params in a tensor
Count Parameters: When iterating over model parameters, .numel() helps determine the total number of elements in each parameter tensor.
Memory Estimation: Knowing the number of elements allows you to estimate the memory required for storing the tensor.
Debugging: Helps ensure the expected number of elements in a tensor, especially when dealing with reshaping or slicing operations.


LoRA: Fine-tunes only a small subset of parameters (low-rank adapters) while freezing most of the original model parameters.
This function helps verify that the model is correctly set up for LoRA by ensuring only the intended parameters are trainable.
During LoRA setup, you'd expect the trainable% to be very small (e.g., <1%), as most parameters remain frozen.
'''

"\nWhy Use .numel()? : Returns total number of params in a tensor\nCount Parameters: When iterating over model parameters, .numel() helps determine the total number of elements in each parameter tensor.\nMemory Estimation: Knowing the number of elements allows you to estimate the memory required for storing the tensor.\nDebugging: Helps ensure the expected number of elements in a tensor, especially when dealing with reshaping or slicing operations.\n\n\nLoRA: Fine-tunes only a small subset of parameters (low-rank adapters) while freezing most of the original model parameters.\nThis function helps verify that the model is correctly set up for LoRA by ensuring only the intended parameters are trainable.\nDuring LoRA setup, you'd expect the trainable% to be very small (e.g., <1%), as most parameters remain frozen.\n"

In [9]:
from peft import LoraConfig , get_peft_model

config  = LoraConfig(
    r =16 ,# attention heads
    lora_alpha = 32 ,   #alpha scaling
    # For llama models : target_modules  = ["q_proj","v_proj"]
    # In bloom we can use : target_modules = ["query_key_value"]
    # target_values = ["query_key_value"]
    lora_dropout = 0.05 ,
    bias = "none",
    # Bloom is a causal language model , hence set task as causal lm
    task_type = "CAUSAL_LM" # set this for CLM or Seq2seq
)
# get a new model with the ,
#
model = get_peft_model(model,config)
print_trainable_parameters(model)

# LoraConfig: Defines the configuration for LoRA fine-tuning, specifying the settings for how the low-rank adaptation is applied
# get_peft_model: A utility function to apply LoRA to an existing model, integrating the LoRA parameters into the model's architecture.
'''
r(RANK)=16 -> Number of compositions we are going to limit our decomposed matrices to .
Rank by initial dimension and initial dimension by rank - two decomposed matrices
Specifies the rank of the low-rank matrices used in LoRA.
In LoRA, instead of training the full parameter matrices of the model, a pair of low-rank matrices (rank r) is trained and added to the original weights.
Higher r values allow the model to learn more complex adaptations but require more compute and memory.

lora_alpha=32   -> lora_alpha is a hyperparameter that controls the strength of the low-rank adaptation. This is usually double of what rank is

A scaling factor applied to the LoRA updates.
Controls how much influence the LoRA layers have on the overall model's behavior.
Larger values of lora_alpha scale the LoRA updates more aggressively.

Target_values : LORA can be applied to any weight matrix , and here we can choose query_key_value as the weight matrix for the attention mechanism

target_modules=["q_proj", "v_proj"]
Specifies which modules in the model are targeted for LoRA adaptation.
For transformer-based models:
q_proj refers to the query projection in the attention mechanism.
v_proj refers to the value projection.
By focusing only on specific modules, LoRA reduces the number of trainable parameters and computation cost.

lora_dropout=0.05
Introduces a small dropout rate to the LoRA updates.
Helps improve generalization by randomly dropping a portion of the LoRA updates during training.

bias="none"
Specifies how the bias terms in the model are handled.
"none": No bias terms are added or trained.
"all": Bias terms are trainable.
"lora_only": Only biases corresponding to LoRA modules are trainable ;


task_type="CAUSAL_LM"
Defines the type of task the model is being fine-tuned for.
"CAUSAL_LM": Indicates the task is causal language modeling (e.g., autoregressive text generation).
Other possible task types include "SEQ2SEQ_LM" (sequence-to-sequence tasks) or "TOKEN_CLASSIFICATION".
'''

trainable params:0 || all_params:256901120 || trainable% :0.0
trainable params:0 || all_params:256902144 || trainable% :0.0
trainable params:0 || all_params:256903168 || trainable% :0.0
trainable params:0 || all_params:256904192 || trainable% :0.0
trainable params:0 || all_params:256905216 || trainable% :0.0
trainable params:0 || all_params:260050944 || trainable% :0.0
trainable params:0 || all_params:260054016 || trainable% :0.0
trainable params:16384 || all_params:260070400 || trainable% :0.00629983266069495
trainable params:65536 || all_params:260119552 || trainable% :0.02519456899572086
trainable params:65536 || all_params:261168128 || trainable% :0.02509341415503809
trainable params:65536 || all_params:261169152 || trainable% :0.025093315768012294
trainable params:65536 || all_params:261170176 || trainable% :0.025093217381758015
trainable params:65536 || all_params:261171200 || trainable% :0.02509311899627524
trainable params:65536 || all_params:265365504 || trainable% :0.02469650

In [ ]:
 # the total trainable parameters are tiny , less than 1%



DATA

In [11]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
# data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

README.md:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [13]:
data

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2508
    })
})

In [14]:
data["train"][:5]

{'quote': ['“Be yourself; everyone else is already taken.”',
  "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”",
  "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”",
  '“So many books, so little time.”',
  '“A room without books is like a body without a soul.”'],
 'author': ['Oscar Wilde',
  'Marilyn Monroe',
  'Albert Einstein',
  'Frank Zappa',
  'Marcus Tullius Cicero'],
 'tags': [['be-yourself',
   'gilbert-perreira',
   'honesty',
   'inspirational',
   'misattributed-oscar-wilde',
   'quote-investigator'],
  ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst'],
  ['human-nature',
   'humor',
   'infinity',
   'philosophy',
   'science',
   'stupidity',
   'universe'],
  ['books', 'humor'],
  ['books', 'simile', 'soul']]}

In [18]:
# FINE-TUNING IS FOR STRUCTURE NOT KNOWLEDGE

# Give a quote by the user , predict its tag
def merge_columns(example):
  # tell the model : every time it sees (->)
  # condition on the input before that , and generate the tags after that
  #
  example["prediction"] = example["quote"] + "->:"+str(example["tags"])
  return example


data["train"] = data["train"].map(merge_columns)
data["train"]["prediction"][3:5]




Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

["“So many books, so little time.”->:['books', 'humor']",
 "“A room without books is like a body without a soul.”->:['books', 'simile', 'soul']"]

In [19]:
data["train"][0]

{'quote': '“Be yourself; everyone else is already taken.”',
 'author': 'Oscar Wilde',
 'tags': ['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator'],
 'prediction': "“Be yourself; everyone else is already taken.”->:['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']"}

In [21]:
data = data.map(lambda samples:tokenizer(samples["prediction"]),batched= True)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [22]:
data
# extra columns : prediction , input_ids , attention_mask




DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 2508
    })
})

TRAINING

In [2]:
trainer = transformers.Trainer(
    model = model ,
    train_dataset = data["train"],
    args = transformers.TrainingArguments(
        per_device_train_batch_size =4,
        gradient_accumulation_steps = 4 ,
    # we set a warmup to start with an extremely low learning rate and
    # build it up from there to 2e-4
        warmup_steps = 100,
        max_steps = 200 ,
        learning_rate = 2e-4 ,
        fp16 = True,
        logging_steps = 1,
        output_dir ="outputs"
    ) ,
  # set mlm equal to false , because bloom is not a mass language model
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer,mlm=False)
)
model.config.use_cache = False  # to silence for warnings , reenable for inference
trainer.train()






NameError: name 'transformers' is not defined

In [ ]:
 # Save the adapters on the HUB -
 # Adpaters are basically ijectable decomposable matrix pairs
 model.push_to_hub("goffer/bloomz-560m-lora",
                   use_auth_token= True,
                   commit_message= "LORA Adapters",
                   private = True
)

INFERENCE - LOAD ADAPTERS FROM THE HUB

In [ ]:
# USE THE MODEL WE SAVED FOR INFERENCE
import torch
from peft import PeftModel , PeftConfig
from transformers import AutoModelForCausalLM , AutoTokenizer

peft_model_id = "goffer/bloomz-560m-lora"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path , return_dict=True , load_in_8_bit =True)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the dataset .
model = PeftModel.from_pretrained(model,peft_model_id)



In [ ]:
# replace the quote "the world is your oyster , with anything you want ,
# and generate the tags for it "

batch = tokenizer("the world is your oyster ->:" , return_tensors='pt')
with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch,max_new_tokens = 50)

print('\n\n',tokenizer.decode(output_tokens[0].skip_special_tokens=True))